In [80]:
import os
from os import environ

import requests
import io

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt
from scipy.stats import norm

In [90]:
cluster_variables=['Route','SEASON','Weekday']

In [106]:
Bcg=pd.read_csv('Spoil upside calculation database with new RPK.csv',sep=';',skiprows=1)

In [107]:
Bcg['Departure Date']=Bcg['Departure Date'].apply(lambda x: pd.to_datetime(x.split('/')[2]+'-'+x.split('/')[1]+'-'+x.split('/')[0]))

In [108]:
Bcg['Sum_Revenues']=Bcg['Sum_Revenues'].apply(lambda x: np.float(x.replace(',','.')))

In [109]:
Bcg=Bcg.loc[Bcg['leg']=='CPH',:]

In [111]:
sorted([pd.to_datetime(x) for x in Bcg.loc[Bcg['SEASON']=='H','Departure Date'].unique()])

[Timestamp('2017-04-08 00:00:00'),
 Timestamp('2017-04-09 00:00:00'),
 Timestamp('2017-04-10 00:00:00'),
 Timestamp('2017-04-21 00:00:00'),
 Timestamp('2017-04-22 00:00:00'),
 Timestamp('2017-04-23 00:00:00'),
 Timestamp('2017-04-25 00:00:00'),
 Timestamp('2017-04-26 00:00:00'),
 Timestamp('2017-05-02 00:00:00'),
 Timestamp('2017-05-03 00:00:00'),
 Timestamp('2017-05-04 00:00:00'),
 Timestamp('2017-05-07 00:00:00'),
 Timestamp('2017-05-08 00:00:00'),
 Timestamp('2017-05-12 00:00:00'),
 Timestamp('2017-05-13 00:00:00'),
 Timestamp('2017-05-14 00:00:00'),
 Timestamp('2017-05-15 00:00:00'),
 Timestamp('2017-05-19 00:00:00'),
 Timestamp('2017-05-20 00:00:00'),
 Timestamp('2017-05-21 00:00:00'),
 Timestamp('2017-05-22 00:00:00'),
 Timestamp('2017-07-07 00:00:00'),
 Timestamp('2017-07-08 00:00:00'),
 Timestamp('2017-07-09 00:00:00'),
 Timestamp('2017-07-10 00:00:00'),
 Timestamp('2017-07-11 00:00:00'),
 Timestamp('2017-07-12 00:00:00'),
 Timestamp('2017-08-05 00:00:00'),
 Timestamp('2017-08-

In [101]:
Bcg.groupby(['Flight Number','Day_of_week','SEASON'])['Sum_Revenues'].mean()

Flight Number  Day_of_week  SEASON
LG-5430        Sun          H         3285.000000
LG-5431        Fri          H         7151.000000
                            M         5828.333333
               Mon          M         3785.000000
               Sun          H         7505.000000
                            M         3807.250000
               Thu          H         3213.000000
                            M         3536.750000
               Tue          M         3005.800000
               Wed          M         3864.400000
LG-5432        Fri          M         4270.000000
               Mon          H         4333.000000
                            M         4061.000000
               Sun          H         9570.000000
                            M         5505.500000
               Thu          M         3526.600000
               Tue          M         3211.800000
               Wed          M         2277.200000
LG-5433        Fri          H         5095.375000
               

In [78]:

Bcg['Route']='LUX'+'-'+Bcg['leg']
Bcg=Bcg.loc[((Bcg['leg']=='CPH')|(Bcg['leg']=='LCY'))&(Bcg['uplift_population']=='YES'),:]

Bcg.index=range(Bcg.shape[0])

Bcg=Bcg[['Departure Date','Route','Flight Number','SEASON','Sum_ASK','Sum_RPK','New RPK']]

Bcg['LF_old']=Bcg['Sum_RPK']/Bcg['Sum_ASK']
Bcg['LF_new']=Bcg['New RPK']/Bcg['Sum_ASK']

Bcg=Bcg[[x for x in Bcg.columns if 'ASK' not in x and 'RPK' not in x]]

Bcg['Departure Date']=Bcg['Departure Date'].apply(lambda x: pd.to_datetime(x.split('/')[2]+'-'+x.split('/')[1]+'-'+x.split('/')[0]))

Bcg['Month']=Bcg['Departure Date'].apply(lambda x: x.month)
Bcg['Weekday']=Bcg['Departure Date'].apply(lambda x: x.weekday())

Bcg=Bcg[[x for x in Bcg.columns if 'LF' not in x]+[x for x in Bcg.columns if 'LF' in x]]

In [73]:


# GROUPINGS

Bcg_Mean=Bcg.groupby(cluster_variables)['LF_old'].mean().reset_index()\
.merge(Bcg.groupby(cluster_variables)['LF_new'].mean().reset_index(),on=cluster_variables)\
.rename(columns={'LF_old': 'LF_old_mean', 'LF_new': 'LF_new_mean'})

Bcg_Var=Bcg.groupby(cluster_variables)['LF_old'].var().reset_index()\
.merge(Bcg.groupby(cluster_variables)['LF_new'].var().reset_index(),on=cluster_variables)\
.rename(columns={'LF_old': 'LF_old_var', 'LF_new': 'LF_new_var'})

Bcg_N=Bcg.groupby(cluster_variables)['LF_old'].size().reset_index()\
.merge(Bcg.groupby(cluster_variables)['LF_new'].size().reset_index(),on=cluster_variables)\
.rename(columns={'LF_old': 'LF_old_n', 'LF_new': 'LF_new_n'})

replace_value_old=np.mean(Bcg_Var['LF_old_var'])
replace_value_new=np.mean(Bcg_Var['LF_new_var'])

Bcg_Var['LF_old_var']=Bcg_Var['LF_old_var'].apply(lambda x: replace_value_old if pd.isnull(x) or x==0 else x)
Bcg_Var['LF_new_var']=Bcg_Var['LF_new_var'].apply(lambda x: replace_value_new if pd.isnull(x) or x==0 else x)

In [75]:
Bcg_Var

,Route,SEASON,Weekday,LF_old_var,LF_new_var
0,LUX-CPH,H,0,0.004688,0.013476
1,LUX-CPH,H,1,0.004688,0.013476
2,LUX-CPH,H,2,0.001733,0.028193
3,LUX-CPH,H,3,0.000408,0.006531
4,LUX-CPH,H,4,0.000230,0.088801
5,LUX-CPH,H,5,0.005624,0.013476
6,LUX-CPH,H,6,0.003673,0.001250
7,LUX-CPH,L,0,0.010453,0.015597
8,LUX-CPH,L,1,0.010235,0.000818
9,LUX-CPH,L,2,0.004688,0.013476


In [98]:
Raw=pd.concat([pd.read_csv('MAR18_CPH.csv',sep=';'),
pd.read_csv('JUN18_CPH.csv',sep=';'),
pd.read_csv('OCT18_CPH.csv',sep=';'),
pd.read_csv('JAN19_CPH.csv',sep=';'),
pd.read_csv('MAR18_LCY.csv',sep=';'),
pd.read_csv('JUN18_LCY.csv',sep=';'),
pd.read_csv('OCT18_LCY.csv',sep=';'),
pd.read_csv('JAN19_LCY.csv',sep=';')])

Raw['Route']=Raw['Sched Airport Depart']+'-'+Raw['Sched Airport Arriva']

Raw['Calendar day']=Raw['Calendar day'].apply(lambda x: pd.to_datetime(x.split('/')[2]+'-'+x.split('/')[1]+'-'+x.split('/')[0]))
Raw['LF_actual']=Raw['Load Factor'].apply(lambda x: float(x.replace(',','.').replace('%',''))/100)
Raw['Direction']=Raw[['Sched Airport Depart','Sched Airport Arriva']].apply(lambda x: 'O' if x[0]=='LUX' and (x[1]=='CPH' or x[1]=='LCY') else 'I',axis=1)

Raw=Raw.rename(columns={'Calendar day': 'Departure Date'})
Raw=Raw[['Departure Date','Route','Flight Number','Direction','LF_actual']]

Raw['Month']=Raw['Departure Date'].apply(lambda x: x.month)
Raw['Weekday']=Raw['Departure Date'].apply(lambda x: x.weekday())

Raw=Raw[[x for x in Raw.columns if 'LF' not in x]+[x for x in Raw.columns if 'LF' in x]]

Raw_Mean=Raw.groupby(cluster_variables)['LF_actual'].mean().reset_index()\
.rename(columns={'LF_actual': 'LF_actual_mean'})
Raw_Var=Raw.groupby(cluster_variables)['LF_actual'].var().reset_index()\
.rename(columns={'LF_actual': 'LF_actual_var'})
Raw_N=Raw.groupby(cluster_variables)['LF_actual'].size().reset_index()\
.rename(columns={'LF_actual': 'LF_actual_n'})

replace_value_actual=np.mean(Raw_Var['LF_actual_var'])

Raw_Var['LF_actual_var']=Raw_Var['LF_actual_var'].apply(lambda x: replace_value_actual if pd.isnull(x) or x==0 else x)

In [99]:
Data=Bcg_Mean.merge(Bcg_Var,on=cluster_variables)\
.merge(Bcg_N,on=cluster_variables)\
.merge(Raw_Mean,on=cluster_variables)\
.merge(Raw_Var,on=cluster_variables)\
.merge(Raw_N,on=cluster_variables)

Data['t_old']=Data[['LF_actual_mean','LF_actual_var','LF_actual_n','LF_old_mean','LF_old_var','LF_old_n']]\
.apply(lambda x: (x[0]-x[3])/np.sqrt((x[1]/x[2])+(x[4]/x[5])),axis=1)

Data['t_new']=Data[['LF_actual_mean','LF_actual_var','LF_actual_n','LF_new_mean','LF_new_var','LF_new_n']]\
.apply(lambda x: (x[0]-x[3])/np.sqrt((x[1]/x[2])+(x[4]/x[5])),axis=1)

Data['Z_larger']=norm.ppf(0.95)
Data['Z_smaller']=-norm.ppf(0.95)

#Data=Data[cluster_variables+['t_old','t_new','Z_larger','Z_smaller']].sort_values(by=['Flight Number','Weekday'])

Data['decision_old']=Data[['t_old','Z_larger','Z_smaller']].apply(lambda x: 'Increase' if x[0]>x[1] else ('Decrease' if x[0]<x[2] else 'Constant'),axis=1)
Data['decision_new']=Data[['t_new','Z_larger','Z_smaller']].apply(lambda x: 'Increase' if x[0]>x[1] else ('Decrease' if x[0]<x[2] else 'Constant'),axis=1)

Data['decision_new']=Data[['LF_old_mean','LF_new_mean','decision_old','decision_new']]\
.apply(lambda x: x[2] if x[0]==x[1] else x[3],axis=1)

Data=Data[cluster_variables+['LF_actual_mean','LF_old_mean','LF_new_mean','decision_old','decision_new']]\
.sort_values(by=cluster_variables)

In [100]:
Data

,Route,Month,Flight Number,Weekday,LF_actual_mean,LF_old_mean,LF_new_mean,decision_old,decision_new
0,LUX-CPH,1,LG-5433,0,0.414250,0.424242,0.946276,Constant,Decrease
1,LUX-CPH,1,LG-5433,1,0.308250,0.363636,0.741055,Constant,Decrease
2,LUX-CPH,1,LG-5433,2,0.302600,0.386364,0.676305,Decrease,Decrease
3,LUX-CPH,1,LG-5433,3,0.357400,0.500000,0.785122,Decrease,Decrease
4,LUX-CPH,1,LG-5433,4,0.436750,0.439394,1.009087,Constant,Decrease
5,LUX-CPH,1,LG-5435,0,0.205250,0.272727,0.943519,Constant,Decrease
6,LUX-CPH,1,LG-5435,3,0.678800,0.385714,0.976196,Increase,Decrease
7,LUX-CPH,1,LG-5437,4,0.479500,0.402814,0.886792,Constant,Decrease
8,LUX-CPH,3,LG-5431,2,0.452000,0.500000,0.700000,Constant,Constant
9,LUX-CPH,3,LG-5431,3,0.411000,0.457143,0.457143,Constant,Constant
